In [63]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

## Labelling of train and test data

In [116]:
import os
df_train=pd.read_excel('trainhiri-rs-new.xlsx',sheet_name='Train')
df_test=pd.read_excel('Test_sample.xlsx')

## Cleaning for train data

In [117]:
import re
def clean_text(text):
    text=text.lower()
#     text=re.sub('\w*\d\w*','', text)
    text=re.sub('\n',' ',text)
    text=re.sub(r"http\S+", "", text)
    text=re.sub('[^A-Za-z0-9]',' ',text)
    return text

df_train['utterance']=df_train['utterance'].apply(lambda x: clean_text(x))
df_train['utterance']=df_train['utterance'].apply(lambda x: re.sub(' +',' ',x))
df_train['utterance']=df_train['utterance'].str.replace(".", "")

# selecting on > 2 len sentence utterances
df_trainutt=df_train["utterance"].values
df_trainutt
s = pd.Series(df_trainutt)
df_train["utt"]=s.str.split().to_frame()
type(s.str.split().to_frame())
df_train['Length'] = df_train['utt'].str.len()
df_train
#last step
df_train.query('Length > 2', inplace = True) 
df_train=df_train[["utterance","intent"]]
df_train

,utterance,intent
0,i am unable to access my profile through dbs ...,onboarding.letter_of_appointment
1,i have not received any notice of new team me...,onboarding.arrangements_for_new_hire_prior_to_...
2,500 iflex top up,iflex.what_can_be_claimed_against_iflex
3,1 lot of dbs share,shares.how_can_i_find_out_the_price_of_a_dbsh_...
5,13 month bonus,payroll.annual_wage_supplement
...,...,...
16569,year end compensation review,compensation.how_to_access_the_year-end_comp_r...
16570,yearly annual leave,leave.can_i_take_leave
16571,yearly bonus crediting date,payroll.when_is_our_bonus_credited
16572,yearly bonus date,payroll.when_is_our_bonus_credited


## Cleaning for test data

In [118]:
import re
def clean_text(text):
    text=text.lower()
#     text=re.sub('\w*\d\w*','', text)
    #text=re.sub('\n',' ',text)
    #text=re.sub(r"http\S+", "", text)
    text=re.sub('[^A-Za-z0-9]',' ',text)
    return text
df_test['utterance']=df_test['utterance'].astype(str)
df_test['utterance']=df_test['utterance'].apply(lambda x: clean_text(x))

df_test['utterance']

df_test['utterance']=df_test['utterance'].apply(lambda x: re.sub(' +',' ',x))
df_test['utterance']=df_test['utterance'].str.replace(".", "")

df_test=df_test[["utterance","intent"]]
df_test

,utterance,intent
0,staff overseas rate,payroll.transfer_my_previous_employer_orso_con...
1,overseas fund transfer for staff,internal_mobility.internal_mobility_eligibilit...
2,carry over leave,leave.can_i_carry_forward_my_leave
3,dbs shares,shares.crediting_of_contribution_shares
4,mobility,internal_mobility.internal_mobility
...,...,...
3790,how do i enrol for ishares,shares.how_to_join_ishares
3791,how do i change the contribution for ishares,shares.can_i_stop_or_change_my_ishares_contrib...
3792,how can i reactivate,future_of_work.do_i_need_to_be_upskilled_or_re...
3793,hi my access has been denied due to password e...,onboarding.new_hire_1bank_password


In [119]:
df_test['utterance'][df_test['utterance']=='']='NONE'

## Labelling

In [120]:
#df=pd.concat([df_train[['utterance','intent']],df_test[['utterance','intent']]])
le=preprocessing.LabelEncoder()
le.fit(df_train['intent'])

df_train['intent1']=le.transform(df_train['intent'])
df_test['intent1']=0

mapping=df_train[['intent','intent1']].drop_duplicates().rename(columns={'intent1':'mapping'})

## Save the files

In [124]:
df_train.to_excel('trainhiri-rs-new-labelled.xlsx',index=None)

In [125]:
writer=pd.ExcelWriter('testhiri-rs-new-labelled.xlsx',engine='xlsxwriter')

In [126]:
df_test.to_excel(writer,sheet_name='Test',index=None)
mapping.to_excel(writer,sheet_name='Mapping',index=None)

In [127]:
writer.save()